In this project, you will put all the skills acquired throughout the course and your knowledge of basic Python to test. You will work on real-world data and perform the operations of Extraction, Transformation, and Loading as required. Throughout the project, you will note some outputs you need to answer questions on the graded quiz. You will also take snapshots, which you will upload in the peer-graded assignment.

#**Project Scenario**
A multi-national firm has hired you as a data engineer. Your job is to access and process data as per requirements.

Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.

**Directions**
1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
    a. Extract the information for the London office, that is Name and MC_GBP_Billion
    b. Extract the information for the Berlin office, that is Name and MC_EUR_Billion
    c. Extract the information for New Delhi office, that is Name and MC_INR_Billion
7. Write a function to log the progress of the code.
8. While executing the data initialization commands and function calls, maintain appropriate log entries.

In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now=datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./code_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')

def extract():
    url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
    try:
        response=requests.get(url,timeout=60)
        response.raise_for_status()
        html_page=response.text
    except Exception as e:
        log_progress(f"Extraction failed: {str(e)}")
        print(f"Error: {e}")
        return pd.DataFrame()
    
    data=BeautifulSoup(html_page,'html.parser')
    df=pd.DataFrame(columns=["Name","MC_USD_Billion"])
    tables=data.find_all('tbody')
    rows=tables[0].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        if len(col) !=0:
            bank_name = col[1].find_all('a')[1].contents[0]
            market_cap = float(col[2].contents[0].strip())

            data_dict = {"Name": bank_name, "MC_USD_Billion": market_cap}
            df1 = pd.DataFrame(data_dict, index=[0]) 
            df = pd.concat([df, df1], ignore_index=True)
    return df 

def transform(df):
    exchange_rate_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

    df_rates = pd.read_csv(exchange_rate_url)
    rates = df_rates.set_index('Currency').to_dict()['Rate']

    df['MC_GBP_Billion'] = [round(x * rates['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [round(x * rates['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [round(x * rates['INR'], 2) for x in df['MC_USD_Billion']]

    return df

def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)

def load_to_db(df,sql_connection,table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    log_progress('Data loaded to Database as table.')

def run_queries(query_statement, sql_connection):
    print(f"Executing Query: {query_statement}")
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)


db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'

#1. Process started log
log_progress('Preliminaries complete. Initiating ETL process')

#2. Extraction step
df = extract()
log_progress('Data extraction complete. Initiating Transformation process')
#3. Transformation step
df=transform(df)
log_progress('Data transformation complete. Initiating loading process')

#4. Load CSV file
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

# 5. Load Database
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated')

load_to_db(df, sql_connection, table_name)

# 6. Run query (Task 6)
# Query 1: Select all data
query_1 = f"SELECT * FROM {table_name}"
run_queries(query_1, sql_connection)

# Query 2: Market Capital average (GBP)
query_2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_queries(query_2, sql_connection)

# Query 3: Select first five bank
query_3 = f"SELECT Name from {table_name} LIMIT 5"
run_queries(query_3, sql_connection)

log_progress('Queries executed. Process Complete.')

# ৭. Close Connection
sql_connection.close()
log_progress('Server Connection closed')


Executing Query: SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31       